In [ ]:
'''
This is for Bigram Model
'''

import torch
import torch.nn.functional as F

words = open("names.txt", 'r').read().splitlines()

N = torch.zeros((27, 27, 27), dtype=torch.int32)


In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
g = torch.Generator().manual_seed(2147483647)

W = torch.rand((27, 27), generator=g, requires_grad=True)

xs, ys = [], []

words_len = len(words)
train_idx = int(0.80 * words_len)
dev_idx = int(0.90 * words_len)


In [4]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        xs.append([ix1, ix2])
        ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

xtrain, ytrain = xs[:train_idx], ys[:train_idx]
xdev, ydev = xs[train_idx:dev_idx], ys[train_idx:dev_idx]
xtest, ytest = xs[dev_idx:], ys[dev_idx:]

In [17]:
num = xs.nelement()

for k in range(100):
    xenc = F.one_hot(xtrain, num_classes=27).float()
    logits = xenc.view(-1, 27) @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    # print(ys.shape[0])
    loss = -probs[torch.arange(ytrain.shape[0]), ytrain].log().mean() + 0.01*(W**2).mean()
    
    W.grad = None
    loss.backward()

    W.data += -0.1 * W.grad

    print(loss.item())

2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623992919921875
2.623993158340454
2.623992681503296
2.623992681503296
2.623992681503296
2.623992681503296
2.623992681503296
2.623992443084717
2.623992443084717
2.623993158340454
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.623992443084717
2.62399244

In [18]:
@torch.no_grad()
def eval(x, y):
    xenc = F.one_hot(x, num_classes=27).float()
    logits = xenc.view(-1, 27) @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    return -probs[torch.arange(y.shape[0]), y].log().mean().item()

train_loss = eval(xdev, ydev)
print("Training loss on training set is, ", train_loss)

Training loss on training set is,  2.6172215938568115


In [19]:
@torch.no_grad()
def eval(x, y):
    xenc = F.one_hot(x, num_classes=27).float()
    logits = xenc.view(-1, 27) @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    return -probs[torch.arange(y.shape[0]), y].log().mean().item()

test_loss = eval(xtest, ytest)
print("Training loss on test set is, ", test_loss)

Training loss on test set is,  2.739518165588379


In [20]:
'''
 If the distance between training score and val score is massive, such as training score is very low and val score is high, it may mean your models started to remember
the index of your training set after you over training it
'''

'\n If the distance between training score and val score is massive, such as training score is very low and val score is high, it may mean your models started to remember\nthe index of your training set after you over training it\n'

In [23]:
N = torch.zeros((27, 27, 27), dtype=torch.int32)

chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoTri = {}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [24]:
chars.insert(0, '.')

i = 0
for ch1 in chars:
    for ch2 in chars:
        stoTri[ch1 + ch2] = i
        i += 1

xs, ys = [], []

words_len = len(words)
train_idx = int(0.80 * words_len)
dev_idx = int(0.90 * words_len)

for w in words:
    chs = ['.'] + ['.'] + list(w) + ['.']

    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoTri[ch1 + ch2]
        ix2 = stoi[ch3]

        xs.append(ix1)
        ys.append(ix2)
    

xs = torch.tensor(xs)
ys = torch.tensor(ys)

xtrain, ytrain = xs[:train_idx], ys[:train_idx]
xdev, ydev = xs[train_idx:dev_idx], ys[train_idx:dev_idx]
xtest, ytest = xs[dev_idx:], ys[dev_idx:]

num = xs.nelement()

W = torch.rand((729, 27), requires_grad=True)

In [25]:
for k in range(100):
    xenc = F.one_hot(xs, num_classes=729).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    # print(ys.shape[0])
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    
    W.grad = None
    loss.backward()

    W.data += -50 * W.grad

    print(loss.item())

3.314655065536499
3.2441201210021973
3.1894381046295166
3.143207311630249
3.1018693447113037
3.0643017292022705
3.0300228595733643
2.998720169067383
2.970123291015625
2.9439659118652344
2.919987201690674
2.8979387283325195
2.877594232559204
2.858754873275757
2.841247797012329
2.8249244689941406
2.809657335281372
2.795337200164795
2.781869649887085
2.7691731452941895
2.757175922393799
2.7458155155181885
2.7350363731384277
2.72478985786438
2.7150321006774902
2.705723762512207
2.696831703186035
2.6883230209350586
2.6801700592041016
2.6723480224609375
2.6648337841033936
2.6576061248779297
2.6506474018096924
2.6439402103424072
2.637468099594116
2.631218194961548
2.6251766681671143
2.6193323135375977
2.6136741638183594
2.608192205429077
2.602876901626587
2.5977203845977783
2.5927140712738037
2.5878515243530273
2.583125114440918
2.5785295963287354
2.5740582942962646
2.5697062015533447
2.5654680728912354
2.561338424682617
2.557314157485962
2.55338978767395
2.5495617389678955
2.5458264350891113

In [32]:
@torch.no_grad()
def eval(x, y):
    xenc = F.one_hot(x, num_classes=729).float()
    logits = xenc.view(-1, 729) @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    return -probs[torch.arange(y.shape[0]), y].log().mean().item()

train_loss = eval(xdev, ydev)
print("Training loss on dev set is, ", train_loss)

Training loss on dev set is,  2.3132517337799072


In [33]:
@torch.no_grad()
def eval(x, y):
    xenc = F.one_hot(x, num_classes=729).float()
    logits = xenc.view(-1, 729) @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    return -probs[torch.arange(y.shape[0]), y].log().mean().item()

train_loss = eval(xtest, ytest)
print("Training loss on test set is, ", train_loss)

Training loss on test set is,  2.452087879180908
